In [1]:
from tensorflow.keras.models import load_model, Sequential,Model 
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy
import pandas as pd
import tensorflow as tf
from PIL import Image
import numpy as np
import pickle

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
with open("xiaoming_dataset_xy_120.pickle", 'rb') as handle:
    (x,y) = pickle.load(handle)
x = x.astype(np.float32)/255

In [3]:
print(x.shape,y.shape)

(5992, 120, 50, 3) (5992,)


In [4]:
print(y)

[1 0 1 ... 0 0 0]


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(5392, 120, 50, 3) (5392,) (600, 120, 50, 3) (600,)


In [10]:
def get_model(input_shape=(100,50,3)):
    model2 = Sequential()
    model2.add(Conv2D(16,(2,2),strides=(1,1),padding = 'same', input_shape=input_shape))
    model2.add(Activation('relu'))
    model2.add(MaxPooling2D(pool_size=(2, 2),padding = 'same'))
    model2.add(Conv2D(32,(2,2),strides=(1,1),padding = 'same'))
    model2.add(Activation('relu'))
    model2.add(MaxPooling2D(pool_size=(2, 2),padding = 'same'))
    model2.add(Conv2D(64,(2,2),strides=(1,1),padding = 'same'))
    model2.add(Activation('relu'))
    model2.add(Flatten())
    model2.add(Dense(64,activation='relu', kernel_initializer='he_normal'))
    model2.add(Dropout(0.3))
    model2.add(Dense(64,activation='relu', kernel_initializer='he_normal'))
    model2.add(Dropout(0.3))
    model2.add(Dense(2,kernel_initializer='he_normal'))
    model2.add(Activation('softmax'))
    return model2

In [11]:

model1 = get_model(input_shape=x_train[0].shape)
sgd = SGD(lr=0.01,momentum=0.0)
adam=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model1.compile(optimizer=adam,
              loss=sparse_categorical_crossentropy,
              metrics=['accuracy'])
model1.fit(x_train,y_train, epochs=12,validation_data=(x_test, y_test))
res=model1.evaluate(x_test,y_test)
print("eval res",res)

Train on 5392 samples, validate on 600 samples
Epoch 1/12
5392/5392 [==============================] - 2s 356us/sample - loss: 0.7000 - acc: 0.4976 - val_loss: 0.6931 - val_acc: 0.4900
Epoch 2/12
5392/5392 [==============================] - 2s 290us/sample - loss: 0.6934 - acc: 0.5085 - val_loss: 0.6932 - val_acc: 0.4983
Epoch 3/12
5392/5392 [==============================] - 2s 289us/sample - loss: 0.6932 - acc: 0.5007 - val_loss: 0.6932 - val_acc: 0.4967
Epoch 4/12
5392/5392 [==============================] - 2s 291us/sample - loss: 0.6932 - acc: 0.4996 - val_loss: 0.6931 - val_acc: 0.4983
Epoch 5/12
5392/5392 [==============================] - 2s 293us/sample - loss: 0.6932 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.4983
Epoch 6/12
5392/5392 [==============================] - 2s 292us/sample - loss: 0.6934 - acc: 0.4991 - val_loss: 0.6932 - val_acc: 0.4983
Epoch 7/12
5392/5392 [==============================] - 2s 290us/sample - loss: 0.6932 - acc: 0.4993 - val_loss: 0.6932 - val

In [12]:
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 120, 50, 16)       208       
_________________________________________________________________
activation_8 (Activation)    (None, 120, 50, 16)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 60, 25, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 60, 25, 32)        2080      
_________________________________________________________________
activation_9 (Activation)    (None, 60, 25, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 13, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 13, 64)       